In [1]:
from DataSelect import *
FOLDER= "/Data"

os.chdir(FOLDER)

CNVfile = 'CNVs.txt'
Somyfile= 'Ploidy.txt'
Transcriptfile = 'RNA.txt'
Proteinfile = 'Proteins.txt'
Outputfile = 'Normalised_multiomics_perchrom.txt'
transdict, samples = readomic(Transcriptfile)
proteindict, samples = readomic(Proteinfile)
ploidydict, samples = readomic(Somyfile)

trans = normalise(dicttopandas(transdict))
protein = dicttopandas(proteindict)

diploidref = {}

for chromosome in ploidydict:
    if chromosome[2:] not in ['31','09','23','37']: #Exclude chromosomes that are never diploid
        diploidref[chromosome] = []
        for sample in samples:
            if 'S' not in sample and 'L' not in sample: #Exclude STAT phase samples and 'L' samples, which are extra LOG samples paired with STAT
                factor = float(ploidydict[chromosome][sample])
                if factor > 1.8 and factor < 2.2: #Tolerance on somy for chromosome to be considered diploid
                    diploidref[chromosome].append([sample,factor])
                    diploidref[chromosome].append([sample.replace('.1','.2'),factor])
                    diploidref[chromosome].append([sample.replace('.1','.3'),factor])
                    diploidref[chromosome].append([sample.replace('.1','.4'),factor])

Groups = ['282', '173', '575', '288', '178', '275']

PloidyFCperchrom = ploidyperchrom(ploidydict)

TransFC = foldchange(Groups, diploidref, trans, datatype="LOG", select='all', randomref = False)
TransFCperchromALL = chromaverager(TransFC, Groups)

ProteinFC = foldchangePT(Groups, diploidref, protein, datatype="else", select='all', randomref = False)
ProteinFCperchromALL = chromaverager(ProteinFC, Groups)

with open(Outputfile, 'w') as mywritefile:
    mywritefile.write("\t".join(
        ["ID", "Chromosome", "Sample", "Somy", "TransAverage",
         "ProteinAverage", "NProteins"]) + '\n')
    for chr in TransFCperchromALL:
        for sample in TransFCperchromALL[chr]:
            mywritefile.write(''.join([str(chr + '_' + sample), '\t', chr, '\t', sample, '\t', transformer(
                [PloidyFCperchrom[chr][sample], TransFCperchromALL[chr][sample][0],
                 ProteinFCperchromALL[chr][sample][0]]), '\t' + str(ProteinFCperchromALL[chr][sample][1]), '\n']))